# Clasificación de municipios (Similares)

Un municipio puede ser “similar” a otro según varias dimensiones, como puede ser su Índice global (aunque esta es demasiado general). En este caso se emplean los vectores de categorías (salud, educación, bienestar, economía…). Municipios similares serán los que tengan perfiles parecidos en estas dimensiones.

Los motivos son los siguientes:

- Están normalizadas (0–1).

- Resumen información real del municipio.

- Son robustas a ruido.

- Son interpretables (cuando veas un cluster sabrás por qué están juntos).

Así que el clustering se hará sobre estas 6 columnas.

In [20]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances

In [23]:
# Cargar el conjunto de datos
df = pd.read_csv("df_indices_categorias.csv")

feature_cols = ["sanidad", "educacion", "transporte_conectividad", "economia_empleo", "bienestar", "ocio_cultura"]
X = df[feature_cols].copy()
municipios = df["municipio"]

print(X.head())

    sanidad  educacion  transporte_conectividad  economia_empleo  bienestar  \
0  0.028502   0.246131                 0.191864         0.191846   0.258489   
1  0.798747   0.850934                 0.885501         0.252383   0.971967   
2  0.428211   0.319385                 0.175599         0.191741   0.258489   
3  0.731336   0.850777                 0.666215         0.284128   0.929250   
4  0.452316   0.503347                 0.464478         0.205005   0.384412   

   ocio_cultura  
0      0.317134  
1      0.508077  
2      0.558106  
3      0.537494  
4      0.551711  


## PCA

- No dependes de clusters.
- Cada municipio tendrá su propio conjunto único de similares,

In [24]:
# PCA (2 O 3 componentes funcionanmuy bien)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Matriz de distancias
dist_matrix = cosine_distances(X_pca)

# Función para obtener los municipios más similares
k = 5  # Número de municipios similares a encontrar

similares = {}

for i, muni in enumerate(municipios):
    # ordenar por distancia (de menor a mayor)
    idx = dist_matrix[i].argsort()[1:k+1]
    similares[muni] = municipios.iloc[idx].tolist()

# Convertimos a tabla
df_similares = (
    pd.DataFrame.from_dict(similares, orient="index")
      .reset_index()
)

df_similares.columns = ["municipio", "sim1", "sim2", "sim3", "sim4", "sim5"]

df_similares.to_csv("municipios_similares.csv", index=False)
df_similares.head()

,municipio,sim1,sim2,sim3,sim4,sim5
0,Acebeda (La),Somosierra,Horcajuelo de la Sierra,Puebla de la Sierra,Robledillo de la Jara,Estremera
1,Ajalvir,Valdeolmos-Alalpardo,Daganzo de Arriba,Torrejón de la Calzada,Villanueva del Pardillo,Cobeña
2,Alameda del Valle,Canencia,Lozoya,Garganta de los Montes,Cenicientos,Villavieja del Lozoya
3,Álamo (El),Valverde de Alcalá,Molar (El),Alpedrete,Becerril de la Sierra,Quijorna
4,Aldea del Fresno,Valdemanco,Puentes Viejas,Olmeda de las Fuentes,Ambite,Berrueco (El)


¿Por qué usar distancia coseno?

Porque es más estable cuando los datos están estandarizados.

Evita que un municipio con valores más grandes parezca “más lejano”

Captura muy bien el “perfil” de un municipio